# Pure Vibe Coding 

## Installation and Imports

In [1]:
%%capture
# Optional: Clean up old numpy to avoid ABI mismatch
!pip uninstall -y numpy

# ✅ Reinstall numpy first to ensure ABI compatibility
!pip install numpy==1.26.4 --force-reinstall

# ✅ Install vLLM and compatible versions
!pip install -U transformers==4.45.2 vllm==0.6.0
!pip install -U torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cu121

# Optional but helpful:
!pip uninstall -y pynvml
!pip install nvidia-ml-py

# For quantized model support:
!pip install optimum auto-gptq

In [2]:
import re
from typing import List, Optional
import torch
import vllm
from vllm import SamplingParams
from collections import defaultdict, Counter
from typing import List

2025-05-03 09:51:19.524516: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746265879.927210      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746265880.057194      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Loading Model and Tokenizer

In [3]:
# model_id = "Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4"
model_id = "Qwen/Qwen2.5-14B-Instruct-AWQ"

llm = vllm.LLM(
    model_id,
    quantization="awq",
    max_model_len=4096,
    enable_prefix_caching=True,
    tensor_parallel_size=torch.cuda.device_count(),
)

tokenizer = llm.get_tokenizer()

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

WARNING 05-03 09:51:41 config.py:330] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 05-03 09:51:41 config.py:890] Defaulting to use mp for distributed inference
INFO 05-03 09:51:41 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='Qwen/Qwen2.5-14B-Instruct-AWQ', speculative_config=None, tokenizer='Qwen/Qwen2.5-14B-Instruct-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=N

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/107k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 05-03 09:52:51 model_runner.py:926] Loading model weights took 4.6719 GB
(VllmWorkerProcess pid=167) INFO 05-03 09:52:54 model_runner.py:926] Loading model weights took 4.6719 GB
INFO 05-03 09:52:57 distributed_gpu_executor.py:57] # GPU blocks: 3965, # CPU blocks: 2730
INFO 05-03 09:53:03 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-03 09:53:03 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
(VllmWorkerProcess pid=167) INFO 05-03 09:53:03 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode

## Defining System Prompt

In [4]:
SYS_PROMPT = """You are an evaluator that checks student answers based on the given question and rubric.

For every input, follow these steps in order:
1. **Evaluation**: Go through each rubric item and assign marks accordingly. 
   - Enclose marks in `<marks>` tags like this: `<marks>1/2</marks>`.
2. **Explanation**: For each rubric item, explain the reasoning behind the assigned marks.
3. **Final Score**: Sum the individual rubric scores and provide a total, using `<score>` tags. 
   - Format: `Final score: <score>7/8</score>`

Always use these three clearly labeled sections:  
`Evaluation:`  
`Explanation:`  
`Final Score:`  

Be objective, fair, and detailed. Do not skip any rubric.

Below are examples for you to learn the format:

**Example 1:**

question = Find the derivative of f(x) = x³ + 3x² + 1.

rubric = "1. States the general power rule (2 points)
2. Applies the power rule to x³ correctly (2 points)
3. Applies the power rule to 3x² correctly (2 points)
4. Simplifies the derivative properly (2 points)

answer = f'(x) = 3x² + 6x

Response:

Evaluation:
1. The student did not explicitly state the general power rule. <marks>0/2</marks>
2. The student correctly applied the power rule to x³. <marks>2/2</marks>
3. The student correctly applied the power rule to 3x². <marks>2/2</marks>
4. The simplification is correct. <marks>2/2</marks>

Explanation:
1. The general rule (d/dx[xⁿ] = n·xⁿ⁻¹) was not stated, though applied correctly.
2. d/dx[x³] = 3x², which the student wrote.
3. d/dx[3x²] = 6x, which is also correct.
4. There are no like terms, and the expression is simplified properly.

Final Score: <score>6/8</score>

**Example 2:**

question = What is the capital of France?

rubric = 1. Identifies the capital correctly (2 points)
2. Provides any additional context (1 point)

answer = Paris is the capital of France. It is known for the Eiffel Tower.

Response:

Evaluation:
1. Correct capital is stated. <marks>2/2</marks>
2. Additional context about Eiffel Tower is provided. <marks>1/1</marks>

Explanation:
1. Paris is indeed the capital of France.
2. Mentioning the Eiffel Tower adds informative context about the city.

Final Score: <score>3/3</score>

"""

## LLM Engine

In [5]:
def llm_engine(
    list_of_messages: List[List[dict]],
    stop_sequences: Optional[List[str]] = None,
    start_sequence: Optional[str] = None,
    temperature: float = 0.2,
    max_tokens: int = 2048
) -> List[str]:
    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=0.9,
        max_tokens=max_tokens,
        stop=stop_sequences,
        include_stop_str_in_output=True,
    )

    prompts = [
        tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        for messages in list_of_messages
    ]

    if start_sequence:
        prompts = [prompt + start_sequence for prompt in prompts]

    outputs = llm.generate(prompts, sampling_params, use_tqdm=False)
    responses = [o.outputs[0].text.strip() for o in outputs]

    if start_sequence:
        responses = [start_sequence + response for response in responses]

    return responses


## Parsing Raw Responses

In [6]:
def parse_evaluation_response(response: str):
    """
    Extracts rubric-wise scores, explanations, and total score from the model's output.
    Assumes the structure:
    Evaluation:
    1. explanation <marks>X/Y</marks>
    2. explanation <marks>X/Y</marks>
    ...
    Explanation:
    1. ...
    2. ...
    Final Score:
    The final score is: <score>X/Y</score>
    """
    # --- Extract rubric-wise scores with explanations ---
    eval_section = re.search(r"Evaluation:(.*?)(Explanation:|Final Score:)", response, re.DOTALL)
    explanation_section = re.search(r"Explanation:(.*?)(Final Score:|$)", response, re.DOTALL)
    final_score = re.search(r"<score>(\d+)\s*/\s*(\d+)</score>", response)

    rubric_scores = []
    if eval_section:
        lines = eval_section.group(1).strip().split("\n")
        for line in lines:
            match = re.search(r"(.*)<marks>(\d+)\s*/\s*(\d+)</marks>", line.strip())
            if match:
                rubric_text = match.group(1).strip()
                score = int(match.group(2))
                total = int(match.group(3))
                rubric_scores.append((rubric_text, score, total))

    explanations = []
    if explanation_section:
        lines = explanation_section.group(1).strip().split("\n")
        explanations = [line.strip() for line in lines if line.strip()]

    total_score = None
    if final_score:
        total_score = (int(final_score.group(1)), int(final_score.group(2)))

    return rubric_scores, explanations, total_score


### Batchwise Parsing

In [7]:
def parse_batch_evaluations(responses: list[str]):
    """
    Processes a list of model responses, extracting:
    - Rubric-wise evaluations (text + marks)
    - Explanations
    - Final scores
    Returns a list of dicts, one per response.
    """
    results = []

    for idx, response in enumerate(responses):
        rubric_scores, explanations, total_score = parse_evaluation_response(response)

        entry = {
            "response_index": idx,
            "rubric_scores": rubric_scores,       # List of (rubric_text, score, total)
            "explanations": explanations,         # List of explanation strings
            "final_score": total_score            # Tuple (score, total) or None
        }

        results.append(entry)

    return results


### Helper function to make messages for LLM

In [8]:
def make_messages(question: str, answer: str, rubric: str) -> list[dict]:
    return [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": f"""QUESTION:
{question}

STUDENT ANSWER:
{answer}

RUBRIC:
{rubric}
"""}
    ]

## Function for Majority Voting

In [9]:
def majority_vote_with_explanations(parsed_batch):
    """
    Takes parsed batch outputs and returns majority-voted rubric scores with explanations.

    Returns:
    - List of tuples: (rubric_text, voted_score, total, explanation)
    """
    rubric_scores = defaultdict(list)  # i -> [(score, total)]
    rubric_explanations = defaultdict(list)  # i -> [explanation]
    rubric_texts = []

    for entry in parsed_batch:
        for i, (rubric_text, score, total) in enumerate(entry['rubric_scores']):
            rubric_scores[i].append((score, total))
            rubric_explanations[i].append(entry['explanations'][i])
            if len(rubric_texts) <= i:
                rubric_texts.append(rubric_text)

    results = []
    for i in range(len(rubric_scores)):
        # Majority vote on score
        score_counter = Counter(rubric_scores[i])Screenshot from 2025-05-03 17-05-41
        (voted_score, voted_total), _ = score_counter.most_common(1)[0]

        # Most common explanation
        explanation_counter = Counter(rubric_explanations[i])
        voted_explanation, _ = explanation_counter.most_common(1)[0]

        results.append((rubric_texts[i], voted_score, voted_total, voted_explanation))

    return results


## For testing Individually

In [21]:
question = "A car initially at rest, achieved a speed of 60 km/h in a minute. Calculate the accelaration of the car"
answer = """Given u=0m/s,t=60s
v=60/3.6=16.67m/s
a=0.278ms^-2
"""
rubric = """1. Identifies the given informations correctly (3 points)
2. Mentions the formula for accelaration explicitely (2 points)
3. Uses the correct formula for accelaration (3 points)
4. Calculates the correct accelaration(Might be in m/s or km/h).(2 points)
"""

In [13]:
question = """
0 থেকে π পর্যন্ত y=sinx দ্বারা আবদ্ধ ক্ষেত্রফল কত?
"""
answer = """
ধরি, 
\[
A = \int_{0}^{\pi} \sin x \, dx
\]

আমরা জানি,
\[
\int \sin x \, dx = -\cos x + C
\]

অতএব,
\[
A = \left[-\cos x\right]_{0}^{\pi}
= \left(-\cos \pi\right) - \left(-\cos 0\right)
\]

\[
= -(-1) - (-1)
= 1 + 1 = 2
\]

সুতরাং,
\[
\boxed{ \text{ক্ষেত্রফল} = 2 \text{ একক} }
\]
"""

rubric = """
1. ক্ষেত্রফলের সাথে সম্পর্কিত নির্দিষ্ট ইন্টিগ্রাল চিহ্নিত করে। (4 points)
2. ক্ষেত্রফল ইন্টিগ্রেট করার জন্য সঠিক সূত্র ব্যবহার করে। (3 points)
3. সঠিক ক্ষেত্রফল গণনা করে। (3 points)
"""

In [11]:
# question = "Find the derivative of f(x) = x³ + 3x² + 1."
# answer = "f'(x) = 3x² + 6x."
# rubric = """1. States the general power rule (d/dx[xⁿ]=n·xⁿ⁻¹) (2 points)
# 2. Applies the power rule correctly to x³ (2 points)
# 3. Applies the power rule correctly to 3x² (2 points)
# 4. Simplifies and combines like terms properly (2 points)"""

messages_batch = [make_messages(question, answer, rubric)] * 10  # for 5 voting samplesScreenshot from 2025-05-03 17-05-41

responses = llm_engine(messages_batch)
parsed_results = parse_batch_evaluations(responses)
# majority_scores,majority_explanations,totals = majority_scores_per_rubric(parsed_results)
results = majority_vote_with_explanations(parsed_results)
total_mark = 0

# for result in parsed_results:
#     print(f"\n🔎 Response #{result['response_index'] + 1}")

#     marks_got = 0
#     total_marks = 0
#     for i, ((rubric, score, total), explanation) in enumerate(zip(result["rubric_scores"], result["explanations"]), 1):
#         marks_got += score
#         total_marks += total
#         print(f"{i}. {rubric} => Score: {score}/{total}")
#         print(f"   Explanation: {explanation}")
    
#     if result["final_score"]:
#         print(f"\n✅ Final Score: {result['final_score'][0]}/{result['final_score'][1]}")
#     total_mark = total_marks
#     print(f"\nSummed Final Score: {marks_got}/{total_marks}\n")

majority_scores = []
for rubric_text, score, total, explanation in results:
    majority_scores.append(score)
    total_mark += total
    print(f"{rubric_text}. Score => {score}/{total}")
    # print(f"Explanation: {explanation}")
    print()


print(f"\nFinal Score according to majority voting: {sum(majority_scores)}/{total_mark}")

1. The student correctly identifies the integral for the area under the curve \( y = \sin x \) from \( 0 \) to \( \pi \).. Score => 4/4

2. The student uses the correct formula for integrating \(\sin x\), which is \(\int \sin x \, dx = -\cos x + C\).. Score => 3/3

3. The student correctly calculates the area under the curve, which is 2 square units.. Score => 3/3


Final Score according to majority voting: 10/10


### The following cell prints the raw responses

In [12]:
# question = "Find the derivative of f(x) = x³ + 3x² + 1."
# answer = "f'(x) = 3x² + 6x."
# rubric = """1. States the general power rule (d/dx[xⁿ]=n·xⁿ⁻¹) (2 points)
# 2. Applies the power rule correctly to x³ (2 points)
# 3. Applies the power rule correctly to 3x² (2 points)
# 4. Writes the final result properly(no need to write 0 in the final result) (2 points)"""

messages_batch = [make_messages(question, answer, rubric)] * 5  # for 5 voting samples

responses = llm_engine(messages_batch)

for response in responses:
    print(response)

Evaluation:
1. The student correctly identifies the integral for the area under the curve \( y = \sin x \) from \( 0 \) to \( \pi \). <marks>4/4</marks>
2. The student uses the correct formula for integrating \( \sin x \), which is \( \int \sin x \, dx = -\cos x + C \). <marks>3/3</marks>
3. The student calculates the area correctly, resulting in an area of 2 square units. <marks>3/3</marks>

Explanation:
1. The student correctly sets up the integral \( A = \int_{0}^{\pi} \sin x \, dx \), which is the integral for the area under the curve \( y = \sin x \) from \( 0 \) to \( \pi \).
2. The student uses the correct antiderivative \( \int \sin x \, dx = -\cos x + C \) to evaluate the definite integral.
3. The student evaluates the definite integral correctly, resulting in \( A = 2 \) square units.

Final Score: <score>10/10</score>
Evaluation:
1. The student correctly identifies the integral for the area under the curve \( y = \sin x \) from \( 0 \) to \( \pi \). <marks>4/4</marks>
2. The

# Evaluating Multiple questions at once!

In [14]:
questions_answers_list = [
    {
        "question" : """
        0 থেকে π পর্যন্ত y=sinx দ্বারা আবদ্ধ ক্ষেত্রফল কত?
        """,
        "answer" : """
        ধরি, 
        \[
        A = \int_{0}^{\pi} \sin x \, dx
        \]
        
        আমরা জানি,
        \[
        \int \sin x \, dx = -\cos x + C
        \]
        
        অতএব,
        \[
        A = \left[-\cos x\right]_{0}^{\pi}
        = \left(-\cos \pi\right) - \left(-\cos 0\right)
        \]
    
        \[
        = -(-1) - (-1)
        = 1 + 1 = 2
        \]
        
        সুতরাং,
        \[
        \boxed{ \text{ক্ষেত্রফল} = 2 \text{ একক} }
        \]
        """,
    
        "rubric" : """
        1. ক্ষেত্রফলের সাথে সম্পর্কিত নির্দিষ্ট ইন্টিগ্রাল চিহ্নিত করে। (4 points)
        2. ক্ষেত্রফল ইন্টিগ্রেট করার জন্য সঠিক সূত্র ব্যবহার করে। (3 points)
        3. সঠিক ক্ষেত্রফল গণনা করে। (3 points)
        """
    },
    {
        "question" : "A car initially at rest, achieved a speed of 60 km/h in a minute. Calculate the accelaration of the car",
        "answer" : """Given u=0m/s,t=60s
        v=60/3.6=16.67m/s
        a=0.278ms^-2
        """,
        "rubric" : """1. Identifies the given informations correctly (3 points)
        2. Mentions the formula for accelaration explicitely (2 points)
        3. Uses the correct formula for accelaration (3 points)
        4. Calculates the correct accelaration(Might be in m/s or km/h).(2 points)
        """
    },
    {
        "question" : """The time period of a simple pendulum in earth is 2 seconds. What will be its time period
        in space?
        """,
        "answer" : "The time period will be unchanged. It will still be 2 seconds",
        "rubric" : "1. Provides the correct answer (2 points)"
    }
]

In [16]:
messages_batches = []
for qa in questions_answers_list: # This list is a list of dictionaries(for now)
    question, answer, rubric = qa["question"], qa["answer"], qa["rubric"]
    single_message = make_messages(question, answer, rubric)
    batch = [single_message] * 5  # 5 copies per question
    messages_batches.extend(batch)  # flatten for vllm batch

## Function to run LLM for batches

In [17]:
def chunked(iterable, size):
    """Yield successive chunks from iterable."""
    for i in range(0, len(iterable), size):
        yield iterable[i:i + size]

def run_llm_in_batches(messages_batches: List[list], batch_size: int = 48, group_size: int = 5) -> List[List[str]]:
    """
    Runs the llm_engine in fixed-size batches and groups responses for majority voting.

    Args:
        messages_batches (List[list]): List of chat-format messages (prompt per instance).
        batch_size (int): Number of messages to send per call to llm_engine.
        group_size (int): Number of completions per logical instance (for majority voting).

    Returns:
        grouped_responses (List[List[str]]): Responses grouped per question.
    """
    all_responses = []

    for batch in chunked(messages_batches, batch_size):
        responses = llm_engine(batch)
        all_responses.extend(responses)

    # Group responses (e.g., for 5 votes per question)
    grouped_responses = [all_responses[i:i + group_size] for i in range(0, len(all_responses), group_size)]

    return grouped_responses

## Function for Majority voting in Grouped Responses

In [18]:
def aggregate_majority_votes(grouped_responses):
    """
    Parses and performs majority voting with explanations on grouped LLM responses.

    Args:
        grouped_responses (List[List[str]]): Responses grouped per question.

    Returns:
        List[List[Tuple[str, int, int, str]]]: 
            For each question:
                A list of tuples (rubric_text, majority_score, total, explanation).
    """
    final_results = []

    for group in grouped_responses:
        parsed = parse_batch_evaluations(group)
        rubric_scores = majority_vote_with_explanations(parsed)
        final_results.append(rubric_scores)

    return final_results

## Helper function to print the final Evaluation

In [19]:
def print_majority_vote_results(final_results, questions=None):
    """
    Nicely prints rubric-wise majority scores and explanations.

    Args:
        final_results: List of lists of tuples (rubric_text, score, total, explanation)
        questions: Optional list of question texts for headers
    """
    for i, rubric_scores in enumerate(final_results):
        if questions:
            print(f"\n📌 Question {i+1}: {questions[i]}")
        else:
            print(f"\n📌 Question {i+1}:")

        total_score = 0
        total_possible = 0

        for j, (rubric, score, total, explanation) in enumerate(rubric_scores):
            print(f"\n🧾 Rubric {j+1}: {rubric}")
            print(f"✅ Score: {score}/{total}")
            print(f"🗒️ Explanation: {explanation}")
            total_score += score
            total_possible += total

        print(f"\n🏁 Final Score: {total_score}/{total_possible}")
        print("-" * 50)

In [20]:
grouped_outputs = run_llm_in_batches(messages_batches, batch_size=48, group_size=5)
final_results = aggregate_majority_votes(grouped_outputs)
print_majority_vote_results(final_results)


📌 Question 1:

🧾 Rubric 1: 1. The student correctly identifies the integral for the area under the curve \( y = \sin x \) from \( 0 \) to \( \pi \).
✅ Score: 4/4
🗒️ Explanation: 1. The student correctly sets up the integral \( A = \int_{0}^{\pi} \sin x \, dx \) to find the area under the curve.

🧾 Rubric 2: 2. The student uses the correct formula for integrating \( \sin x \), which is \( \int \sin x \, dx = -\cos x + C \).
✅ Score: 3/3
🗒️ Explanation: 2. The student correctly uses the antiderivative \( \int \sin x \, dx = -\cos x + C \) to evaluate the definite integral.

🧾 Rubric 3: 3. The student correctly calculates the definite integral and finds the area to be 2 square units.
✅ Score: 3/3
🗒️ Explanation: 3. The student correctly evaluates the definite integral and finds the area to be 2 square units.

🏁 Final Score: 10/10
--------------------------------------------------

📌 Question 2:

🧾 Rubric 1: 1. The student correctly identifies the given information: initial velocity (u=0 